### Determining actual neighbours in deskewed image using affine matrices

When deskewing, the actual neighbours are in the intermediated sheared image

We use trilinear interpolation by first determining neighbours in 3D, x0,y0,z0 to x8,y8,z8
from the sheared image. Once we find the neighbours, we apply an inverse shear transform to find the position of these neighbours in the raw data and interpolate these points

https://en.wikipedia.org/wiki/Trilinear_interpolation

In [2]:
import math
import pyclesperanto_prototype as cle
from pyclesperanto_prototype._tier8._affine_transform import _determine_translation_and_bounding_box
from skimage.io import imread
import numpy as np 
#Disable scientific notation for numbers
np.set_printoptions(suppress=True)

voxel_size_x_in_microns = 0.1449922
voxel_size_y_in_microns = 0.1449922
voxel_size_z_in_microns = 0.3

deskewing_angle_in_degrees = 30

original_image = imread("C:\\Users\\deepu\\OneDrive - wehi.edu.au\\WEHI_projects\\Lightsheet\\pyclesperanto_prototype\\data\\RBC_tiny.tif")
original_image = original_image[::4]
voxel_size_z_in_microns = voxel_size_z_in_microns * 4
print(original_image.shape)


#Defining Deskewing transform 

scale_factor = 1 #Scale factor is used in clesperanto if we want to scale the images in case they don't fit in memory

#initialise a transform
deskew_transform =cle.AffineTransform3D()
#Shear
shear_factor = math.sin((90 - deskewing_angle_in_degrees) * math.pi / 180.0) * (voxel_size_z_in_microns/voxel_size_y_in_microns)
deskew_transform._matrix[1, 2] = shear_factor
#Scale
new_dz = math.sin(deskewing_angle_in_degrees * math.pi / 180.0) * voxel_size_z_in_microns
scale_factor_z = (new_dz / voxel_size_y_in_microns) * scale_factor
deskew_transform.scale(scale_x=scale_factor, scale_y=scale_factor, scale_z=scale_factor_z)
#Rotate deskewing angle
deskew_transform.rotate(angle_in_degrees = 0 - deskewing_angle_in_degrees, axis=0)
_,deskew_transform,translation = _determine_translation_and_bounding_box(original_image, deskew_transform)

deskew_transform._matrix
deskewed = cle.affine_transform(source= original_image,transform = deskew_transform,auto_size=True)
deskewed = cle.pull(deskewed)
print(deskewed.shape)

(209, 118, 209)
(59, 1832, 209)


In [3]:
#Get shear transform
shear_transform =cle.AffineTransform3D()
#Shear
shear_factor = math.sin((90 - deskewing_angle_in_degrees) * math.pi / 180.0) * (voxel_size_z_in_microns/voxel_size_y_in_microns)
shear_transform._matrix[1, 2] = shear_factor
print(shear_transform._matrix)

#Inverse shear transform
shear_inverse = shear_transform.copy().inverse()
print(shear_inverse._matrix)


[[1.         0.         0.         0.        ]
 [0.         1.         7.16749235 0.        ]
 [0.         0.         1.         0.        ]
 [0.         0.         0.         1.        ]]
[[ 1.          0.          0.          0.        ]
 [ 0.          1.         -7.16749235  0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          1.        ]]


Have to determine how the neighbours are defined!

Currently, using translations in y and z to get neighbours

### Trilinear interpolation?
Get neighbours in yz plane
* (y+1,z-1)
* (y-1,z-1)
* (y-1,z+1)
* (y+1,z+1)

Get neighbours in xy plane
* (x-1,y+1)
* (x-1,y-1)
* (x+1,y-1)
* (x+1,y+1)

In [10]:
#Lets say we want to find neighbours for coordinate in deskewed image (xyz): 70, 464, 21
#
coord = [70,464,21,1]
print("Deskewed image coordinates are:", coord)

#Apply inverse deskew to find corresponding coordinaes in original image
deskew_inv_coord = deskew_transform.copy().inverse()._matrix@coord
print("On raw image, coordinates are:", deskew_inv_coord)

#Apply shear transform to find coordinates in sheared image
sheared_img_coord = shear_transform._matrix@deskew_inv_coord
print("On intermediate sheared image, coordinates are:", sheared_img_coord)


#get 8 coordinates



#Find neighbours in the yz plane for the sheared image
transform_xyz1= cle.AffineTransform3D()
transform_xyz2= cle.AffineTransform3D()
transform_xyz3= cle.AffineTransform3D()
transform_xyz4= cle.AffineTransform3D()


#Find neighbours in the xy plane for the sheared image
transform_xyz5= cle.AffineTransform3D()
transform_xyz6= cle.AffineTransform3D()
transform_xyz7= cle.AffineTransform3D()
transform_xyz8= cle.AffineTransform3D()


#Need to figure out the right value to use as step size?
#For not, this is to find adjacent neighbours in 3D space
transform_xyz1.translate(translate_x=-1,translate_y=1,translate_z=-1)
transform_xyz2.translate(translate_x=-1,translate_y=-1,translate_z=-1)
transform_xyz3.translate(translate_x=1,translate_y=-1,translate_z=-1)
transform_xyz4.translate(translate_x=1,translate_y=1,translate_z=-1)

transform_xyz5.translate(translate_x=-1,translate_y=1,translate_z=1)
transform_xyz6.translate(translate_x=-1,translate_y=-1,translate_z=1)
transform_xyz7.translate(translate_x=1,translate_y=-1,translate_z=1)
transform_xyz8.translate(translate_x=1,translate_y=1,translate_z=1)


xyz1_sheared = transform_xyz1._matrix@sheared_img_coord
xyz2_sheared = transform_xyz2._matrix@sheared_img_coord

xyz3_sheared = transform_xyz3._matrix@sheared_img_coord
xyz4_sheared = transform_xyz4._matrix@sheared_img_coord


xyz5_sheared = transform_xyz5._matrix@sheared_img_coord
xyz6_sheared = transform_xyz6._matrix@sheared_img_coord

xyz7_sheared = transform_xyz7._matrix@sheared_img_coord
xyz8_sheared = transform_xyz8._matrix@sheared_img_coord


print("\nSheared image neighbour coordinates plane 1")
print("bottom left", xyz1_sheared)
print("top left",xyz2_sheared)
print("top right",xyz3_sheared)
print("bottom right",xyz4_sheared)



print("Sheared image neighbour coordinates plane 2")
print("bottom left", xyz5_sheared)
print("top left",xyz6_sheared)
print("top right",xyz7_sheared)
print("bottom right",xyz8_sheared)


#get inverse shear transform
shear_transform_inverse = shear_transform.copy().inverse()


xyz1 = shear_transform_inverse._matrix@xyz1_sheared
xyz2 = shear_transform_inverse._matrix@xyz2_sheared
xyz3 = shear_transform_inverse._matrix@xyz3_sheared
xyz4 = shear_transform_inverse._matrix@xyz4_sheared


xyz5 = shear_transform_inverse._matrix@xyz5_sheared
xyz6 = shear_transform_inverse._matrix@xyz6_sheared
xyz7 = shear_transform_inverse._matrix@xyz7_sheared
xyz8 = shear_transform_inverse._matrix@xyz8_sheared

print("\nActual neighbours in the raw image plane 1")

print("bottom left", xyz1)
print("top left",xyz2)
print("top right",xyz3)
print("bottom right",xyz4)

print("Actual neighbours in the raw image plane 2")

print("bottom left", xyz5)
print("top left",xyz6)
print("top right",xyz7)
print("bottom right",xyz8)



Deskewed image coordinates are: [70, 464, 21, 1]
On raw image, coordinates are: [70.         76.         48.11107853  1.        ]
On intermediate sheared image, coordinates are: [ 70.         420.83578736  48.11107853   1.        ]

Sheared image neighbour coordinates plane 1
bottom left [ 69.         421.83578736  47.11107853   1.        ]
top left [ 69.         419.83578736  47.11107853   1.        ]
top right [ 71.         419.83578736  47.11107853   1.        ]
bottom right [ 71.         421.83578736  47.11107853   1.        ]
Sheared image neighbour coordinates plane 2
bottom left [ 69.         421.83578736  49.11107853   1.        ]
top left [ 69.         419.83578736  49.11107853   1.        ]
top right [ 71.         419.83578736  49.11107853   1.        ]
bottom right [ 71.         421.83578736  49.11107853   1.        ]

Actual neighbours in the raw image plane 1
bottom left [69.         84.16749235 47.11107853  1.        ]
top left [69.         82.16749235 47.11107853  1.    

In [11]:


#Bilinear interpolation

#Lets say we want to find neighbours for coordinate in deskewed image (xyz): 70, 464, 21
#
coord = [70,464,21,1]
print("Deskewed image coordinates are:", coord)

#Apply inverse deskew to find corresponding coordinaes in original image
deskew_inv_coord = deskew_transform.copy().inverse()._matrix@coord
print("On raw image, coordinates are:", deskew_inv_coord)

#Apply shear transform to find coordinates in sheared image
sheared_img_coord = shear_transform._matrix@deskew_inv_coord
print("On intermediate sheared image, coordinates are:", sheared_img_coord)


#Find neighbours in the yz plane for the sheared image
transform_yz1= cle.AffineTransform3D()
transform_yz2= cle.AffineTransform3D()
transform_yz3= cle.AffineTransform3D()
transform_yz4= cle.AffineTransform3D()

y_displacement = voxel_size_z_in_microns/np.float32(np.tan(deskewing_angle_in_degrees * np.pi/180))

transform_yz1.translate(translate_y= -y_displacement,translate_z=-1)
transform_yz2.translate(translate_y=-y_displacement)

transform_yz3.translate(translate_y=y_displacement,translate_z=1)
transform_yz4.translate(translate_y=y_displacement)


yz1_sheared = transform_yz1._matrix@sheared_img_coord
yz2_sheared = transform_yz2._matrix@sheared_img_coord

yz3_sheared = transform_yz3._matrix@sheared_img_coord
yz4_sheared = transform_yz4._matrix@sheared_img_coord

print("Sheared image neighbour coordinates (actual neighbours)")
print("top left ", yz1_sheared)
print("top right",yz2_sheared)
print("bottom left",yz3_sheared)
print("bottom right",yz4_sheared)

#get inverse shear transform
shear_transform_inverse = shear_transform.copy().inverse()


yz1 = shear_transform_inverse._matrix@yz1_sheared
yz2 = shear_transform_inverse._matrix@yz2_sheared
yz3 = shear_transform_inverse._matrix@yz3_sheared
yz4 = shear_transform_inverse._matrix@yz4_sheared


print("Actual neighbours in the raw image")

print("top left ", yz1)
print("top right",yz2)
print("bottom left",yz3)
print("bottom right",yz4)


Deskewed image coordinates are: [70, 464, 21, 1]
On raw image, coordinates are: [70.         76.         48.11107853  1.        ]
On intermediate sheared image, coordinates are: [ 70.         420.83578736  48.11107853   1.        ]
Sheared image neighbour coordinates (actual neighbours)
top left  [ 70.         418.75732635  47.11107853   1.        ]
top right [ 70.         418.75732635  48.11107853   1.        ]
bottom left [ 70.         422.91424836  49.11107853   1.        ]
bottom right [ 70.         422.91424836  48.11107853   1.        ]
Actual neighbours in the raw image
top left  [70.         81.08903135 47.11107853  1.        ]
top right [70.         73.92153899 48.11107853  1.        ]
bottom left [70.         70.91096865 49.11107853  1.        ]
bottom right [70.         78.07846101 48.11107853  1.        ]
